In [ ]:
import json
import os

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from PIL import Image
from pycocotools.coco import COCO
import skimage.io

%load_ext autoreload
%autoreload 2
!nvidia-smi

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"
pd.options.display.max_colwidth = 500

In [ ]:
# load relevant file paths for left & right frames and bbox predictions

data_directory_base = '/root/alok/data/'
stereo_frame_pairs_directory = os.path.join(data_directory_base, 'annotated_stereo_frame_pairs_2')

left_image_file_paths, right_image_file_paths = [], []
left_predictions_paths, right_predictions_paths = [], []
for directory_name in os.listdir(stereo_frame_pairs_directory):
    directory_path = os.path.join(stereo_frame_pairs_directory, directory_name)
    left_image_file_path = os.path.join(directory_path, 'input', 'left_frame.jpg')
    right_image_file_path = os.path.join(directory_path, 'input', 'right_frame.jpg')
    left_predictions_path = os.path.join(directory_path, 'input', 'left_frame.jpg.segmentation.predictions.json')
    right_predictions_path = os.path.join(directory_path, 'input', 'right_frame.jpg.segmentation.predictions.json')
    
    left_image_file_paths.append(left_image_file_path)
    right_image_file_paths.append(right_image_file_path)
    left_predictions_paths.append(left_predictions_path)
    right_predictions_paths.append(right_predictions_path)
    
    
    

<h1> Get statistics on body part annotations across fish </h1>

In [ ]:
df = pd.DataFrame()
for left_predictions_path, right_predictions_path, left_image_file_path, right_image_file_path in \
    zip(left_predictions_paths, right_predictions_paths, left_image_file_paths, right_image_file_paths):
    for side, predictions_path, image_file_path in zip(['left', 'right'], \
                                                       [left_predictions_path, right_predictions_path], \
                                                       [left_image_file_path, right_image_file_path]):
        coco = COCO(predictions_path)
        categories = coco.cats
        annotation_ids = coco.getAnnIds()
        annotations = coco.loadAnns(annotation_ids)
        row = {i: 0 for i in range(1, 10)}
        for annotation in annotations:
            row[annotation['category_id']] += 1
        row['predictions_path'] = predictions_path
        row['image_file_path'] = image_file_path
        row['side'] = side
        df = df.append(row, ignore_index=True)
        

In [ ]:
features = [1, 2, 3, 4, 5, 6, 7, 8, 9]
df

In [ ]:
def plot_annotations(image_file_path, predictions_path):
    coco = COCO(predictions_path)
    image_ids = coco.getImgIds()
    image_data = coco.loadImgs(image_ids[0])[0]

    # load and display instance annotations
    image = skimage.io.imread(image_file_path)
    f, ax = plt.subplots(1, figsize=(20, 20))
    ax.imshow(image)
    ax.axis('off')
    annotation_ids = coco.getAnnIds(imgIds=image_data['id'], iscrowd=None)
    annotations = coco.loadAnns(annotation_ids)
    coco.showAnns(annotations, draw_bbox=True)
    for ann in annotations:
        bbox = ann['bbox']
        rec = patches.Rectangle((bbox[1], bbox[0]), bbox[3]-bbox[1], bbox[2]-bbox[0],
                                edgecolor='w', facecolor=None, fill=False, linestyle='--',
                                linewidth=2)
        ax.add_patch(rec)
        category_id = ann['category_id']
        ax.text(bbox[1], bbox[0] - 10, category_id, fontsize=16, color='w')

In [ ]:
idx = 0
mask = df[1] == 1
plot_annotations(df[mask].image_file_path.iloc[idx], df[mask].predictions_path.iloc[idx])